# TF Dataset을 이용한 데이터 처리 해보기

### import packages

In [148]:
import pathlib
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import tensorflow as tf
np.set_printoptions(precision=3, suppress=True)

### data Load

In [149]:
TRAIN_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/train.csv"
TEST_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/eval.csv"

train_file_path = tf.keras.utils.get_file("train.csv", TRAIN_DATA_URL)
test_file_path = tf.keras.utils.get_file("eval.csv", TEST_DATA_URL)

In [191]:
pd.read_csv(train_file_path).count()

survived              627
sex                   627
age                   627
n_siblings_spouses    627
parch                 627
fare                  627
class                 627
deck                  627
embark_town           627
alone                 627
dtype: int64

In [150]:
LABEL_COLUMN = 'survived'
LABELS = [0, 1]

In [151]:
def get_dataset(file_path, **kwargs):
    dataset = tf.data.experimental.make_csv_dataset(
        file_path, # 파일 경로
        batch_size=5, # 데이터 배치 크기
        label_name=LABEL_COLUMN, # 라벨 이름 설정
        na_value="?", # 
        num_epochs=1,
        ignore_errors=True,
        **kwargs)
    return dataset

raw_train_data = get_dataset(train_file_path)
raw_test_data = get_dataset(test_file_path)

In [188]:
def show_batch(dataset):
    for batch, label in dataset.take(1):
        for key, value in batch.items():
            print("{:20s}: {}".format(key, value.numpy()))
            
def show_all_batch(dataset):
    i = 1
    for batch, label in dataset:
        print('='*10,i,'='*10)
        i+=1
        for key, value in batch.items():
            print("{:20s}: {}".format(key, value.numpy()))

In [153]:
show_batch(raw_train_data)

sex                 : [b'male' b'female' b'female' b'female' b'male']
age                 : [28. 28. 37. 25. 19.]
n_siblings_spouses  : [0 0 0 1 0]
parch               : [0 0 0 0 0]
fare                : [56.496  7.229  9.587  7.925  7.896]
class               : [b'Third' b'Third' b'Third' b'Third' b'Third']
deck                : [b'unknown' b'unknown' b'unknown' b'unknown' b'unknown']
embark_town         : [b'Southampton' b'Cherbourg' b'Southampton' b'Southampton' b'Southampton']
alone               : [b'y' b'y' b'y' b'n' b'y']


### data preprocessing

In [154]:
CSV_COLUMNS = ['survived', 'sex', 'age', 'n_siblings_spouses', 'parch', 'fare', 'class', 'deck', 'embark_town', 'alone']

temp_dataset = get_dataset(train_file_path, column_names = CSV_COLUMNS)

show_batch(temp_dataset)

sex                 : [b'female' b'male' b'female' b'male' b'female']
age                 : [24. 28. 28. 24. 28.]
n_siblings_spouses  : [0 0 0 0 0]
parch               : [0 0 0 0 1]
fare                : [13.     7.896  7.75  79.2   55.   ]
class               : [b'Second' b'Third' b'Third' b'First' b'First']
deck                : [b'F' b'unknown' b'unknown' b'B' b'E']
embark_town         : [b'Southampton' b'Southampton' b'Queenstown' b'Cherbourg' b'Southampton']
alone               : [b'y' b'y' b'y' b'y' b'n']


In [155]:
SELECT_COLUMN = ['survived', 'age', 'n_siblings_spouses', 'parch', 'fare']
DEFAULTS = [0, 0.0, 0.0, 0.0, 0.0]
temp_dataset = get_dataset(train_file_path,
                          select_columns=SELECT_COLUMN,
                          column_defaults=DEFAULTS)
show_batch(temp_dataset)

age                 : [28. 40. 17. 28. 64.]
n_siblings_spouses  : [0. 1. 1. 0. 1.]
parch               : [0. 4. 0. 0. 4.]
fare                : [ 56.496  27.9    57.      7.312 263.   ]


In [156]:
example_batch, labels_batch = next(iter(temp_dataset))

In [157]:
def pack(features, label):
    return tf.stack(list(features.values()), axis=-1), label

In [158]:
packed_dataset = temp_dataset.map(pack)

for features, labels in packed_dataset.take(1):
    print(features.numpy())
    print()
    print(labels.numpy())

[[20.     1.     1.    15.742]
 [ 2.     0.     1.    10.462]
 [35.     1.     0.    53.1  ]
 [28.     0.     0.     7.75 ]
 [28.     0.     0.    13.   ]]

[1 0 1 0 0]


In [159]:
show_batch(raw_train_data)

sex                 : [b'male' b'male' b'male' b'female' b'male']
age                 : [30. 32. 28. 35. 57.]
n_siblings_spouses  : [1 0 1 0 0]
parch               : [0 0 0 0 0]
fare                : [24.   10.5  16.1  21.   12.35]
class               : [b'Second' b'Second' b'Third' b'Second' b'Second']
deck                : [b'unknown' b'unknown' b'unknown' b'unknown' b'unknown']
embark_town         : [b'Cherbourg' b'Southampton' b'Southampton' b'Southampton' b'Queenstown']
alone               : [b'n' b'y' b'n' b'y' b'y']


In [160]:
example_batch, labels_batch = next(iter(temp_dataset))

In [161]:
class PackNumericFeatures(object):
    def __init__(self, names):
        self.names=names
    def __call__(self, features, labels):
        numeric_features = [features.pop(name) for name in self.names]
        numeric_features = [tf.cast(feat, tf.float32) for feat in numeric_features]
        numeric_features = tf.stack(numeric_features, axis=-1)
        features['numeric'] = numeric_features
        
        return features, labels

In [162]:
NUMERIC_FEATURES = ['age', 'n_siblings_spouses', 'parch', 'fare']
packed_train_data =  raw_train_data.map(PackNumericFeatures(NUMERIC_FEATURES))
packed_test_data = raw_test_data.map(PackNumericFeatures(NUMERIC_FEATURES))

In [163]:
show_batch(packed_train_data)

sex                 : [b'male' b'male' b'female' b'male' b'male']
class               : [b'First' b'Third' b'First' b'Second' b'Second']
deck                : [b'C' b'unknown' b'unknown' b'unknown' b'unknown']
embark_town         : [b'Cherbourg' b'Southampton' b'Cherbourg' b'Southampton' b'Southampton']
alone               : [b'n' b'y' b'y' b'n' b'n']
numeric             : [[ 50.      1.      0.    106.425]
 [ 20.      0.      0.      9.225]
 [ 35.      0.      0.    512.329]
 [ 25.      1.      0.     26.   ]
 [ 42.      1.      0.     27.   ]]


In [164]:
example_batch, labels_batch = next(iter(packed_train_data))

In [165]:
desc = pd.read_csv(train_file_path)[NUMERIC_FEATURES].describe()
desc

,age,n_siblings_spouses,parch,fare
count,627.000000,627.000000,627.000000,627.000000
mean,29.631308,0.545455,0.379585,34.385399
std,12.511818,1.151090,0.792999,54.597730
min,0.750000,0.000000,0.000000,0.000000
25%,23.000000,0.000000,0.000000,7.895800
50%,28.000000,0.000000,0.000000,15.045800
75%,35.000000,1.000000,0.000000,31.387500
max,80.000000,8.000000,5.000000,512.329200


In [166]:
MEAN = np.array(desc.T['mean'])
STD = np.array(desc.T['std'])

In [167]:
def normalize_numeric_data(data, mean, std):
    return (data-mean)/std

In [168]:
import functools

normalizer = partial(normalize_numeric_data, mean=MEAN, std=STD)

numeric_column = tf.feature_column.numeric_column('numeric', normalizer_fn=normalizer, shape=[len(NUMERIC_FEATURES)])
numeric_columns = [numeric_column]
numeric_column

NumericColumn(key='numeric', shape=(4,), default_value=None, dtype=tf.float32, normalizer_fn=functools.partial(<function normalize_numeric_data at 0x000001E9C6FD1828>, mean=array([29.631,  0.545,  0.38 , 34.385]), std=array([12.512,  1.151,  0.793, 54.598])))

In [169]:
example_batch['numeric']

<tf.Tensor: shape=(5, 4), dtype=float32, numpy=
array([[28.   ,  0.   ,  0.   , 56.496],
       [28.   ,  1.   ,  2.   , 23.45 ],
       [20.5  ,  0.   ,  0.   ,  7.25 ],
       [23.   ,  0.   ,  0.   , 13.792],
       [31.   ,  0.   ,  0.   , 13.   ]], dtype=float32)>

In [170]:
numeric_layer = tf.keras.layers.DenseFeatures(numeric_columns)
numeric_layer(example_batch).numpy()

array([[-0.13 , -0.474, -0.479,  0.405],
       [-0.13 ,  0.395,  2.043, -0.2  ],
       [-0.73 , -0.474, -0.479, -0.497],
       [-0.53 , -0.474, -0.479, -0.377],
       [ 0.109, -0.474, -0.479, -0.392]], dtype=float32)

In [171]:
CATEGORIES = {
    'sex' : ['male', 'female'],
    'class' : ['First', 'Second', 'Third'],
    'deck' : ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'],
    'embark_town' : ['Cherbourg', 'Southhampton', 'Queenstown'],
    'alone' : ['y', 'n']
}

In [172]:
categorical_columns = []
for feature, vocab in CATEGORIES.items():
    cat_col = tf.feature_column.categorical_column_with_vocabulary_list(
        key=feature, vocabulary_list=vocab)
    categorical_columns.append(tf.feature_column.indicator_column(cat_col))

In [173]:
categorical_columns

[IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='sex', vocabulary_list=('male', 'female'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='class', vocabulary_list=('First', 'Second', 'Third'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='deck', vocabulary_list=('A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='embark_town', vocabulary_list=('Cherbourg', 'Southhampton', 'Queenstown'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='alone', vocabulary_list=('y', 'n'), dtype=tf.string, default_value=-1, num_oov_buckets=0))]

In [174]:
categorical_layer = tf.keras.layers.DenseFeatures(categorical_columns)
print(categorical_layer(example_batch).numpy()[0])

[1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]


In [175]:
preprocessing_layer = tf.keras.layers.DenseFeatures(categorical_columns+numeric_columns)
print(preprocessing_layer(example_batch).numpy()[0])

[ 1.     0.     0.     0.     1.     0.     0.     0.     0.     0.
  0.     0.     0.     0.     0.     0.     0.     0.    -0.13  -0.474
 -0.479  0.405  1.     0.   ]


In [176]:
model = tf.keras.Sequential([
    preprocessing_layer,
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1),
])

model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    optimizer='adam',
    metrics=['accuracy'])

In [177]:
train_data = packed_train_data.shuffle(500)
test_data = packed_test_data

In [178]:
model.fit(train_data, epochs=20)

Epoch 1/20
126/126 [==============================] - 2s 9ms/step - loss: 0.4969 - accuracy: 0.7321
Epoch 2/20
126/126 [==============================] - 1s 7ms/step - loss: 0.4228 - accuracy: 0.8118
Epoch 3/20
126/126 [==============================] - 1s 7ms/step - loss: 0.4000 - accuracy: 0.8198
Epoch 4/20
126/126 [==============================] - 1s 7ms/step - loss: 0.3902 - accuracy: 0.8278
Epoch 5/20
126/126 [==============================] - 1s 7ms/step - loss: 0.3755 - accuracy: 0.8389
Epoch 6/20
126/126 [==============================] - 1s 9ms/step - loss: 0.3707 - accuracy: 0.8469
Epoch 7/20
126/126 [==============================] - 1s 8ms/step - loss: 0.3649 - accuracy: 0.8453
Epoch 8/20
126/126 [==============================] - 1s 7ms/step - loss: 0.3526 - accuracy: 0.8485
Epoch 9/20
126/126 [==============================] - 1s 7ms/step - loss: 0.3578 - accuracy: 0.8405
Epoch 10/20
126/126 [==============================] - 1s 7ms/step - loss: 0.3444 - accuracy: 0.8533

In [179]:
test_loss, test_accuracy = model.evaluate(test_data)

print('\n\nTest Loss {}, Test Accuracy {}'.format(test_loss, test_accuracy))

53/53 [==============================] - 1s 8ms/step - loss: 0.4467 - accuracy: 0.8447


Test Loss 0.44666460156440735, Test Accuracy 0.8446969985961914


In [181]:
predictions = model.predict(test_data)

for prediction, survived in zip(predictions[:10], list(test_data)[0][1][:10]):
    prediction = tf.sigmoid(prediction).numpy()
    print("predicted survival: {:.2%}".format(prediction[0]),
         " | Actual outcome: ",
         ("SURVIVED" if bool(survived) else "DIED"))

predicted survival: 17.12%  | Actual outcome:  DIED
predicted survival: 1.74%  | Actual outcome:  SURVIVED
predicted survival: 100.00%  | Actual outcome:  DIED
predicted survival: 22.91%  | Actual outcome:  SURVIVED
predicted survival: 62.34%  | Actual outcome:  SURVIVED


In [192]:
predictions

array([[-1.577],
       [-4.033],
       [13.221],
       [-1.213],
       [ 0.504],
       [ 1.746],
       [-0.047],
       [-2.697],
       [-2.549],
       [-0.03 ],
       [-2.494],
       [-2.093],
       [-2.381],
       [-1.725],
       [-0.093],
       [-2.987],
       [-1.181],
       [-1.14 ],
       [ 0.199],
       [-2.418],
       [-2.436],
       [-4.352],
       [-1.276],
       [ 5.838],
       [-2.155],
       [-0.143],
       [-0.584],
       [ 0.189],
       [-2.358],
       [-2.729],
       [ 2.755],
       [-1.537],
       [ 0.342],
       [-3.209],
       [-1.293],
       [-2.52 ],
       [-2.648],
       [-2.746],
       [-0.957],
       [-4.018],
       [ 1.431],
       [ 3.518],
       [11.403],
       [-1.687],
       [-2.509],
       [-2.176],
       [-2.358],
       [-0.551],
       [-1.13 ],
       [ 2.932],
       [ 0.535],
       [-0.018],
       [ 4.524],
       [ 6.655],
       [-1.311],
       [-0.085],
       [-5.53 ],
       [-2.09 ],
       [-2.051